In [ ]:
# Import matplotlib before seaborn
import matplotlib as mpl
import matplotlib.pyplot as plt
import itertools  # for color palette cycling
import re
import pandas as pd
import seaborn as sns
import sys
from cycler import cycler
import seaborn as sns
%matplotlib inline

In [ ]:
from datetime import datetime
import pickle
import timeit

import networkx

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import networkx_helpers as nxh

In [ ]:
! pwd

In [ ]:
edge_file = '/work/rnaseq/pcor_new/Ledoit_Wolf_r4.8xlarge_244GB_memory/ledoit_wolf_precision_cutoff_0.005--26967_genes.tsv'

In [ ]:
#edges = nxh.load_network(edge_file, 2.5)
#print(edges.shape)
#edges.head()


In [ ]:
gff_tsv_path = '/work/m4b_binning/assembly/prokka/contigs/contigs_longer_than_1500bp/contigs_longer_than_1500bp_gffs_concatenated.gff.genes.tsv'
#gff_tsv = nxh.load_gff_tsv(gff_tsv_path)
#gff_tsv.head()

In [ ]:
network = nxh.build_network(edge_file, 2.5, gff_tsv_path)

In [ ]:
date = datetime.today().strftime('%y%m%d')
print(date)
networkx.write_gpickle(network, date + 'ledoit_wolf_26967_genes_2.5percent_network.gpickle')

In [ ]:
! ls

In [ ]:
degree_sequence=sorted(networkx.degree(network).values(),reverse=True) # degree sequence

In [ ]:
len(degree_sequence)

In [ ]:
! mkdir -p results

In [ ]:
degree_sequence=sorted(networkx.degree(network).values(),reverse=True) # degree sequence
#print "Degree sequence", degree_sequence
dmax=max(degree_sequence)

fig, ax = plt.subplots(1,1, figsize=(4, 2.5))
ax.loglog(degree_sequence,'b-',marker='o')
plt.title("Degree rank plot")
plt.ylabel("degree")
plt.xlabel("rank")
fig.savefig('./results/170401_degree_rank_plot.pdf')